# DNN(Deep Neural Network)

## Keras로 비만도 테스트

In [45]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import LabelEncoder, MinMaxScaler, OneHotEncoder
from library.preprocessing import DataFramePreprocessor
from sklearn.model_selection import train_test_split

from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation
from keras.utils import np_utils

In [3]:
bmi = pd.read_csv('data/bmi.csv')
bmi

,label,height,weight
0,normal,188,71
1,fat,161,68
2,thin,178,52
3,fat,136,63
4,normal,145,52
...,...,...,...
19995,thin,163,48
19996,fat,139,70
19997,normal,150,48
19998,normal,189,69


In [4]:
bmi.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000 entries, 0 to 19999
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   label   20000 non-null  object
 1   height  20000 non-null  int64 
 2   weight  20000 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 468.9+ KB


In [6]:
bmi.describe(include='all').T

,count,unique,top,freq,mean,std,min,25%,50%,75%,max
label,20000,3,fat,7673,NaN,NaN,NaN,NaN,NaN,NaN,NaN
height,20000.0,NaN,NaN,NaN,160.09205,23.385464,120.0,140.0,160.0,180.0,200.0
weight,20000.0,NaN,NaN,NaN,57.37765,13.260121,35.0,46.0,57.0,69.0,80.0


## 데이터 변환

In [47]:
numeric_cols = ["height", "weight"]
categorical_cols = ["label"]

In [48]:
ohe = OneHotEncoder()
mms = MinMaxScaler()
dfp = DataFramePreprocessor()

In [49]:
data, tfs = dfp.fit_transform_multiple_transformer(bmi, [ohe, mms], [categorical_cols, numeric_cols])

In [50]:
data

,label_fat,label_normal,label_thin,height,weight
0,0.0,1.0,0.0,0.8500,0.800000
1,1.0,0.0,0.0,0.5125,0.733333
2,0.0,0.0,1.0,0.7250,0.377778
3,1.0,0.0,0.0,0.2000,0.622222
4,0.0,1.0,0.0,0.3125,0.377778
...,...,...,...,...,...
19995,0.0,0.0,1.0,0.5375,0.288889
19996,1.0,0.0,0.0,0.2375,0.777778
19997,0.0,1.0,0.0,0.3750,0.288889
19998,0.0,1.0,0.0,0.8625,0.755556


In [13]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000 entries, 0 to 19999
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   label   20000 non-null  int32  
 1   height  20000 non-null  float64
 2   weight  20000 non-null  float64
dtypes: float64(2), int32(1)
memory usage: 390.8 KB


In [55]:
X_data = data[numeric_cols]
y_data = data.iloc[:, 0:3]

## train, test 분할

In [56]:
X_train, X_test, y_train, y_test = train_test_split(X_data, y_data, test_size=0.25, stratify=y_data)

## 케라스 모델 구성

In [66]:
# 모델 구조 정의하기 --- (※3)
model = Sequential()
model.add(Dense(512, activation='relu', input_shape=(X_data.shape[1],)))
model.add(Dropout(0.2))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(32, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(16, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(y_data.shape[1]))
model.add(Activation('softmax'))
model.compile( loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [67]:
# 데이터 훈련하기 --- (※5)
hist = model.fit(X_train, y_train, epochs=50)

Epoch 1/50
469/469 [==============================] - 3s 3ms/step - loss: 0.2605 - accuracy: 0.8956
Epoch 2/50
469/469 [==============================] - 1s 3ms/step - loss: 0.1187 - accuracy: 0.9592
Epoch 3/50
469/469 [==============================] - 1s 3ms/step - loss: 0.0869 - accuracy: 0.9674
Epoch 4/50
469/469 [==============================] - 1s 3ms/step - loss: 0.0763 - accuracy: 0.9709
Epoch 5/50
469/469 [==============================] - 1s 3ms/step - loss: 0.0783 - accuracy: 0.9723
Epoch 6/50
469/469 [==============================] - 1s 3ms/step - loss: 0.0714 - accuracy: 0.9721
Epoch 7/50
469/469 [==============================] - 1s 3ms/step - loss: 0.0711 - accuracy: 0.9711
Epoch 8/50
469/469 [==============================] - 1s 3ms/step - loss: 0.0591 - accuracy: 0.9761
Epoch 9/50
469/469 [==============================] - 2s 3ms/step - loss: 0.0551 - accuracy: 0.9786
Epoch 10/50
469/469 [==============================] - 1s 3ms/step - loss: 0.0607 - accuracy: 0.9764